Tout d'abord, récupérer les fichiers de séquences génomiques fasta.
Utiliser le script ci-dessous afin d'extraire les séquences chromosomiques de chaque génome dans des fichiers fasta séparés
Modifier l'input_file ainsi que les chromosomes_ids pour chaque génome différent.
Si besoin, installer Biopython

In [ ]:
conda activate environpro

: 

Si utilisation du serveur ifb, aller dans le chemin suivant et charger les modules correspondants :

In [ ]:
cd /shared/home/hnguyen97/ondemand/data/sys/dashboard/batch_connect/sys/jupyter/core/projet_pangenome
module load pggb


In [ ]:
conda install biopython

In [ ]:
cd /home/nguyeho3/Documents/Projet_pangenome

In [ ]:
from Bio import SeqIO
import os

# Nom du fichier FASTA source
input_file = "/home/nguyeho3/Documents/(06-9-28)-GWHEQVJ00000000.genome.fasta"

# Liste des identifiants des chromosomes à extraire
chromosome_ids = [
    "GWHEQVJ00000001",
    "GWHEQVJ00000002",
    "GWHEQVJ00000003",
    "GWHEQVJ00000004",
    "GWHEQVJ00000005",
    "GWHEQVJ00000006",
    "GWHEQVJ00000007",
    "GWHEQVJ00000008",
    "GWHEQVJ00000009",
    "GWHEQVJ00000010",
    
    # Ajoutez les autres identifiants ici
]

# Dossier de sortie
output_folder = "/home/nguyeho3/Documents/chromosomes_(06-9-28)"
os.makedirs(output_folder, exist_ok=True)

# Lecture et extraction des séquences correspondant aux identifiants
for record in SeqIO.parse(input_file, "fasta"):
    if record.id in chromosome_ids:
        # Construire un fichier de sortie basé sur l'identifiant
        output_file = os.path.join(output_folder, f"{record.id}.fasta")
        with open(output_file, "w") as f:
            SeqIO.write(record, f, "fasta")
        print(f"Séquence {record.id} sauvegardée dans {output_file}")
    else:
        print(f"Séquence {record.id} ignorée (non dans la liste des identifiants).")

print("Extraction terminée.")



Utiliser le script ci-dessous pour extraire des loci d'intérêt et les concaténer dans un fichier fasta (voir supplementary table 10 pour les position start et end)

In [ ]:
from Bio import SeqIO

# Dossier contenant les fichiers FASTA pour chaque chromosome
fasta_folder = "/home/nguyeho3/Documents/Projet_pangenome/chromosomes/chromosomes_2/"

# Liste des loci à extraire
# Format : {chromosome_filename: [(start1, end1), (start2, end2), ...]}
loci_to_extract = {
    "GWHEQVE00000002.fasta": [(500001, 700000)],
    "GWHEQVJ00000002.fasta": [(500001, 700000)],
    "GWHEQVM00000002.fasta": [(500001,700000)],
    "GWHEQVN00000002.fasta": [(500001,700000)],
    # Ajouter d'autres fichiers et coordonnées si nécessaire
}

# Fichier de sortie
output_file = "extracted_loci_3.fasta"

# Extraire les loci depuis les fichiers FASTA individuels
with open(output_file, "w") as out_file:
    for fasta_file, loci in loci_to_extract.items():
        fasta_path = fasta_folder + fasta_file
        
        try:
            # Charger la séquence du fichier FASTA
            record = next(SeqIO.parse(fasta_path, "fasta"))
            sequence = record.seq
        except FileNotFoundError:
            print(f"Avertissement : Fichier '{fasta_file}' introuvable.")
            continue
        except StopIteration:
            print(f"Avertissement : Fichier '{fasta_file}' est vide.")
            continue
        
        # Extraire les loci pour ce fichier
        for i, (start, end) in enumerate(loci, start=1):
            # Extraire le locus (attention à l'index 0 en Python)
            locus_seq = sequence[start-1:end]
            
            # Créer un header unique pour chaque locus
            header = f">{fasta_file}_locus_{start}_{end}_segment_{i}"
            
            # Sauvegarder dans le fichier FASTA
            out_file.write(f"{header}\n")
            out_file.write(str(locus_seq) + "\n")
            print(f"Locus extrait : {header}")

print(f"Extraction terminée. Résultats sauvegardés dans '{output_file}'.")


Utiliser la commande ci-dessous dans le terminal pour concaténer les séquences chromosomiques d'intérêt en un seul fichier fasta 

In [ ]:
cat GWHEQVE00000008.fasta GWHEQVJ00000008.fasta GWHEQVM00000008.fasta GWHEQVN00000008.fasta > concatChrom1.fa

Utiliser la commande ci-dessous dans le terminal pour utiliser pggb sur la séquence

In [ ]:
nohup pggb -i concatChrom1.fa -n 4 -o Results_PGGB6 --multiqc

Les résultats seront rangés dans le dossier Results_PGGB4

In [ ]:
samtools faidx extracted_loci_2.fasta